## Learning Objectives:

- Modeling of a Single Depot capacitated vehicle routing problem with time windows (CVRPTW) 
- Introduction to more advanced real world constraints like mixed fleet vehicles

### Problem Statement

**The company is now dealing with the following problem:**
- 91 locations to deliver:
    - 1 Depot (Distribution Center)
    - 90 Orders
    - Each location has a specific time window within which services/deliveries need to be performed
    - Demand for orders may differ across locations
- 10 Vehicles in the Fleet
    - Each vehicle has associated capacities
    - Each vehicle has a time window of its availability

In [ ]:
# DO NOT CHANGE THIS CELL
# import dependencies

import cudf
import cupy
import pandas as pd
import numpy as np

from scipy.spatial import distance
from cuopt.routing import utils
from cuopt import routing

import helper_function.helper_map as helper_map
from helper_function.helper_data import parse_output_summary, prepare_data_for_maps
from helper_function.data_etl import get_minutes_from_datetime, build_distance_matrix, build_distance_matrix_osrm, build_time_matrix, plot_order_locations
from helper_function.map_util import get_route, get_map, get_all_routes, get_map_by_vehicle

## Prepare Data

##### cuOpt usage conventions:
1. If there are <mark>n</mark> depots in the use case, you need to add them as top <mark>n</mark> rows in dataframe with <mark>service_time</mark>= 0. cuOpt considers that the vehicles first visit the depot (in this case NV Wholesale) to pick up the orders prior to delivering it to any of the customers (in this case NV Marts).


2. Vehicles are assumed to be starting from the depot in this notebook, however, the vehicle locations can be specified to the cuOPT solver (as latitudes and longitudes) if the vehicles were to start from different locations

In [ ]:
# DO NOT CHANGE THIS CELL

DATA_PATH = "../data/"

# load data

orders_details = pd.read_csv(DATA_PATH+"orders_single_depot.csv")
depot_df = pd.read_csv(DATA_PATH+"single_depot_location.csv")

# ADD COMMENT HERE
orders_details = pd.concat([depot_df, orders_details]).reset_index(drop=True)

# 90 orders + 1 depot = 91 
orders_details = orders_details[:91]

orders_details.head()

In [ ]:
# DO NOT CHANGE THIS CELL
# Creating a 2-dimensional matrix from the columns "lat" and "lng" columns

location_coordinates = orders_details[['lat', 'lng']]
# Output looks like [[32.7727664105, -96.5800824456], [29.6832825242, -95.7305722105]............]


# Units of products to be delivered at each location, this can be number of orders, weight or volume.
orders = orders_details['order_wt']
# 'orders' looks like [1020, 1989.......................2974]

### Prepare Time Windows

The order must be delivered to the customers in specific time windows, these time windows must be provided to the solver as minutes from midnight as start and end times. 

For example a start time of 8:00am would be provided as 60x8 = 480 minutes, similarly an end time of 5:00pm would be 60x17 = 1020 minutes

In [ ]:
# DO NOT CHANGE THIS CELL

#Converting the delivery start and end times to minutes
orders_details['delivery_start_in_minutes'] = orders_details['delivery_start'].apply(get_minutes_from_datetime)
orders_details['delivery_end_in_minutes'] = orders_details['delivery_end'].apply(get_minutes_from_datetime)

# Earliest a delivery can be made
order_tw_earliest = orders_details['delivery_start_in_minutes']
# Latest a delivery can be made
order_tw_latest = orders_details['delivery_end_in_minutes']

# Service time required for the delivery/service
service_time = orders_details['service_time']

### Heterogenous Fleet Routing

In scenarios such as food delivery, the delivery fleet may consist of various types of vehicles, for example bikes, and cars, and each type of vehicle has own advantages and limitations. For example, in crowded streets of NYC, it might be faster to reach a nearby destination on bike compared to car, while it is much faster with car in suburban areas. Service providers can improve customer satisfaction, reduce costs, and increase earning opportunity for drivers, using various types of vehicles depending on the geographical location of the service area.

Logistics industry is moving towards sustinability. Reducing carbon footprint to deliver orders and still maintain supply chain efficiency has become a prominent challenge. Some companies like [Amazon](https://www.aboutamazon.com/news/transportation/amazons-electric-delivery-vehicles-from-rivian-roll-out-across-the-u-s) and [Walmart](https://www.caranddriver.com/news/a40587886/walmart-canoo-ev-delivery-trucks-purchase/) have started adopting EV trucks for delivery.

Similarly vehicle start and end times can be modeled as follows:

In [ ]:
# DO NOT CHANGE THIS CELL

#Similarly, converting the vehicle start and end times to minutes
vehicle_df = pd.read_csv(DATA_PATH+"vehicles_single_depot.csv")
vehicle_df['vehicle_start_in_minutes'] = vehicle_df['vehicle_start'].apply(get_minutes_from_datetime)
vehicle_df['vehicle_end_in_minutes'] = vehicle_df['vehicle_end'].apply(get_minutes_from_datetime)
vehicle_df['vehicle_type_codes'] = vehicle_df['vehicle_type'].astype('category').cat.codes
vehicle_df.head()

In [ ]:
# DO NOT CHANGE THIS CELL

n_locations = len(location_coordinates)
n_vehicles = len(vehicle_df)
n_orders = len(orders_details)
n_depot = 1

# Number of orders each vehicle can carry, this can be number of orders, weight or volumne
vehicle_capacity = vehicle_df['vehicle_capacity']

# Earliest a vehicle can start 
v_tw_earliest = vehicle_df['vehicle_start_in_minutes']

# Latest a vehicle will be working
v_tw_latest = vehicle_df['vehicle_end_in_minutes']

# Used to identify different vehicle
vehicle_colors = ["blue", "white", "green", "pink", "yellow"]

### Visualize Depot and Destinations

This is an interactive map based on <mark>folium</mark> python mapping library, following is the color scheme used for the map.

Depot: 🔴
 <br />Restaurant: 🟢
 <br />Retailer: 🔵
 <br />Business:  🟠

In [ ]:
# DO NOT CHANGE THIS CELL

plot_order_locations(orders_details, location_coordinates)

The **cost matrix** models the cost between each pair of locations.  It is used by NVIDIA cuOpt to compute the cost of traveling from any location to any other.  Here we are going to specify that distance traveled is the cost we are looking to minimize. 

Let's create a distance based cost matrix (called `distance_matrix`)from the location coordinates dataframe, and let the distance between locations be measured by a `euclidean` metric. This will result in a symmetric distance matrix [ distance(A, B) == distance(B, A) ] which is an ideal case. However, in general NVIDIA cuOpt also supports asymmetric matrices which is especially useful when dealing with real-world problems.

In practical applications, you can integrate the distance matrix to a third-party map data provider like Google Maps to get live traffic data and run dynamic/real-time re-routing using <mark>cuOpt</mark>.

In [ ]:
# DO NOT CHANGE THIS CELL

distance_matrix = build_distance_matrix(orders_details)
time_matrix_truck = build_time_matrix(orders_details)
time_matrix_ev = build_time_matrix(orders_details, avg_vehicle_speed = 55)

**Note:** The three functions we have used so far <mark>get_minutes_from_datetime, build_distance_matrix, build_time_matrix</mark> have been curated for this notebook to help us learn quickly, they are not part of the cuOpt SDK.

To compute asymmetric distance matrices we use [OSRM](http://project-osrm.org/docs/v5.22.0/api/?language=Python#general-options)

Note: Please uncomment the cell below to test out the solution with the asymmetric matrix.

In [ ]:
# DO NOT CHANGE THIS CELL

# distance_matrix = build_distance_matrix_osrm(orders_details)
# distance_matrix

In [ ]:
# DO NOT CHANGE THIS CELL
# convert all data into cuDF dataframes

distance_matrix = cudf.DataFrame(distance_matrix)
time_matrix_truck = cudf.DataFrame(time_matrix_truck)
time_matrix_ev = cudf.DataFrame(time_matrix_ev)

## Fleet Data

Setup mixed fleet data

In [ ]:
# type 0 corresponds to truck and type 1 corresponds to ev-van
vehicle_types = cudf.Series(vehicle_df['vehicle_type_codes'])

# bikes can carry two units of goods while car can carry 5 units of goods
vehicle_capacity = cudf.Series(vehicle_df['vehicle_capacity'])

<br>

## Create Data-Model
---
Create a Data model with the following:
 - Number of locations
 - Number of vehicles in the fleet
 - Cost matrix
 - Location time windows
 - Vehicle time windows
 - Vehicle capacities
 - Variable demand across locations

### Initialize routing.DataModel object

In [ ]:
# DO NOT CHANGE THIS CELL

data_model = routing.DataModel(n_locations, n_vehicles)

### Set Vehicle types and corresponding cost matrices

In [ ]:
# DO NOT CHANGE THIS CELL

# set matrices associated with each vehicle type
#data_model.set_vehicle_types(vehicle_types)

# Distance Matrix
data_model.add_cost_matrix(distance_matrix.copy(deep=True))
#data_model.add_cost_matrix(distance_matrix, 1)

# Time matrix
data_model.add_transit_time_matrix(time_matrix_truck)
#data_model.add_transit_time_matrix(time_matrix_ev, 0)

In [ ]:
# DO NOT CHANGE THIS CELL

data_model.add_capacity_dimension(
    "order_wt",
    cudf.Series(orders),
    cudf.Series(vehicle_capacity)
)

data_model.set_order_time_windows(
    cudf.Series(order_tw_earliest),
    cudf.Series(order_tw_latest), 
)

data_model.set_order_service_times(
    cudf.Series(service_time)
)

data_model.set_vehicle_time_windows(
    cudf.Series(v_tw_earliest), 
    cudf.Series(v_tw_latest)
)

# Set minimum number of vehciles that need to be used to compute results
data_model.set_min_vehicles(5)

<br>

## Create Solver Instance
---
The solver instance will take the data-model and return an optimized route plan. Additional configuration options are available to further customize solver behavior including: 
- The number of parallel agents (climbers) examining the solution search space
- The maximum time allotted to find a solution
- The minimum number of vehicles to be used etc.

In [ ]:
# DO NOT CHANGE THIS CELL

solver_settings = routing.SolverSettings()
# set number of climbers that will try to search for an optimal path parallely
solver_settings.set_number_of_climbers(128)
# solver will run for given time limit and it will fail if needs more time
solver_settings.set_time_limit(5)
# solver will drop infeasible orders and assgin remaining orders
solver_settings.set_drop_infeasible_orders(True)

routing_solution = routing.Solve(data_model, solver_settings)
if routing_solution.get_status() == 0:
    print("Solution Found")
else:
    print("No Solution Found")

### Let us have a look at the output

In [ ]:
# DO NOT CHANGE THIS CELL

routes_df = routing_solution.get_route().to_pandas()
routes_df.head()

# Not a looking good, Eh?

## Parse Output

In [ ]:
# DO NOT CHANGE THIS CELL
print("Cost for the routing in time: ", routing_solution.final_cost)
output = parse_output_summary(routing_solution, n_orders, n_vehicles, orders_details, vehicle_df, location_coordinates, distance_matrix, time_matrix_truck, n_depot)

In [ ]:
# DO NOT CHANGE THIS CELL

plot_map_df = prepare_data_for_maps(depot_df, output)
all_routes = get_all_routes(plot_map_df)

##### Let’s visualize our results

**Note :** Folium is used to visualize the routes and OSRM is used to get the asymmetric distance matrix. Both Folium and OSRM are not part of cuOpt SDK. OSRM can be replaced with third-party map frameworks like Google Maps, ESRI maps etc. We are not making a comparison of cuOpt against OSRM as routing solution.

In [ ]:
# DO NOT CHANGE THIS CELL

from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
from ipywidgets import interact

w = widgets.Dropdown(
    options=list(vehicle_df.vehicle_id),
    value=list(vehicle_df.vehicle_id)[0],
    description='Vehicle ID:',
)

def on_change(value):
    if value in output.keys():
        idx = list(plot_map_df[plot_map_df['vehicle_id'] == value].index)
        filtered_dict = {k: v for k, v in all_routes.items() if k in idx}
        display(get_map_by_vehicle(filtered_dict))
        
    else:
        print("This Vehicle is not assigned to any order!!")

interact(on_change, value=w)

## Multi-Depot VRP

We use similar data and constraints but to accomplish solution for Multi-Depot Vehicle Routing Problem(MDVRP)
<mark>Orders, Vehicles and Depot</mark> data will be provided and you need to complete exercise.

### Problem Statement

**The company is now dealing with the following problem:**
- 96 locations to deliver:
    - 3 Depots (Distribution Centers)
    - 93 Orders
    - Each location has a specific time window within which services/deliveries need to be performed
    - Demand for orders may differ across locations
- 10 Vehicles in the Fleet
    - Each vehicle has associated capacities
    - Each vehicle has a time window of its availability

## Prepare Data

##### Remember?
If there are <mark>n</mark> depot in the use case, you need to add as them top <mark>n</mark> rows in dataframe with <mark>service duration</mark> as 0. cuOpt considers first visiting depot as an order to complete.

In [ ]:
# DO NOT CHANGE THIS CELL
# load data

orders_details_md = pd.read_csv(DATA_PATH+"orders_multi_depot.csv")
depot_df_md = pd.read_csv(DATA_PATH+"multi_depot_location.csv")

# ADD COMMENT HERE
orders_details_md = pd.concat([depot_df_md, orders_details_md]).reset_index(drop=True)

orders_details_md = orders_details_md[:96]

orders_details_md.head()


In [ ]:
# DO NOT CHANGE THIS CELL
# Creating a 2-dimensional matrix from the columns "lat" and "lng" columns

location_coordinates_md = orders_details_md[['lat', 'lng']]
# Output looks like [[32.7727664105, -96.5800824456], [29.6832825242, -95.7305722105]............]


# Units of products to be delivered at each location, this can be number of orders, weight or volume.
orders_md = orders_details_md['order_wt']
# 'orders' looks like [1020, 1989.......................2974]


### Prepare Time Windows

In [ ]:
# DO NOT CHANGE THIS CELL
orders_details_md['delivery_start_in_minutes'] = orders_details_md['delivery_start'].apply(get_minutes_from_datetime)
orders_details_md['delivery_end_in_minutes']  = orders_details_md['delivery_end'].apply(get_minutes_from_datetime)


# Earliest a delivery can be made
order_tw_earliest_md = orders_details_md['delivery_start_in_minutes']
# Latest a delivery can be made
order_tw_latest_md = orders_details_md['delivery_end_in_minutes']

# Service time required for the delivery/service
service_time_md = orders_details_md['service_time']


In [ ]:
# DO NOT CHANGE THIS CELL
vehicle_df_md = pd.read_csv(DATA_PATH+"vehicles_multi_depot.csv")
vehicle_df_md['vehicle_start_in_minutes']     = vehicle_df_md['vehicle_start'].apply(get_minutes_from_datetime)
vehicle_df_md['vehicle_end_in_minutes']       = vehicle_df_md['vehicle_end'].apply(get_minutes_from_datetime)
vehicle_df_md.head()

In [ ]:
# DO NOT CHANGE THIS CELL
# Creating a new column with the latitude and longitude as a tuple 
depot_df_md['lat_lng'] = depot_df_md.apply(lambda row: (row['lat'], row['lng']), axis=1)
orders_details_md['lat_lng'] = orders_details_md.apply(lambda row: (row['lat'], row['lng']), axis=1)
depot_df_md.head()

In [ ]:
# DO NOT CHANGE THIS CELL
# Assigning random depots as the vehicles' start locations
random_values = np.random.choice(np.array(depot_df_md['lat_lng']), size=len(vehicle_df_md))

vehicle_df_md['lat_lng'] = pd.Series(random_values)
vehicle_df_md[['lat', 'lng']] = vehicle_df_md['lat_lng'].apply(lambda x: pd.Series(x))
vehicle_df_md

In [ ]:
# DO NOT CHANGE THIS CELL
n_depot              = len(depot_df_md)
n_locations          = len(location_coordinates_md)
n_vehicles           = len(vehicle_df_md)
n_orders             = len(orders_details_md)

# Number of orders each vehicle can carry, this can be number of orders, weight or volumne
vehicle_capacity_md     = vehicle_df_md['vehicle_capacity']

# Earliest a vehicle can start 
v_tw_earliest_md        = vehicle_df_md['vehicle_start_in_minutes']

# Latest a vehicle will be working
v_tw_latest_md          = vehicle_df_md['vehicle_end_in_minutes']

# Used to identify differnt vehicle
vehicle_colors       = ["blue", "white", "green", "pink", "yellow"]

In [ ]:
# DO NOT CHANGE THIS CELL
vehicle_df_md['idx_lat'] = 0

for i in range(n_depot):
    vehicle_df_md.loc[vehicle_df_md['lat'] == orders_details_md.iloc[i]['lat'], 'idx_lat'] = i
vehicle_df_md

In [ ]:
vehicle_locations = cudf.Series(list(vehicle_df_md.idx_lat))
vehicle_locations

### Visualize Depot and Destinations

Depot: 🔴
 <br />Restaurant: 🟢
 <br />Retailer: 🔵
 <br />Business:  🟠

In [ ]:
plot_order_locations(orders_details_md, location_coordinates_md)

In [ ]:
distance_matrix = build_distance_matrix(orders_details_md)
time_matrix = build_time_matrix(orders_details_md)

In [ ]:
# convert all data into cuDF dataframes
distance_matrix = cudf.DataFrame(distance_matrix)
time_matrix = cudf.DataFrame(time_matrix)
location_coordinates_md = cudf.DataFrame(location_coordinates_md[n_depot:])

In [ ]:
# cudf.Series(list(location_coordinates_md.index))

<br>

## Create Data-Model
---
Create a Data model with the following:
 - Number of locations
 - Number of vehicles in the fleet
 - Cost matrix
 - Location time windows
 - Vehicle time windows
 - Vehicle capacities
 - Variable demand across locations

In [ ]:
# DO NOT CHANGE THIS CELL

data_model = routing.DataModel(n_locations, n_vehicles, n_orders-n_depot)
data_model.add_cost_matrix(distance_matrix)
data_model.set_order_locations(cudf.Series(list(location_coordinates_md.index)))
data_model.add_capacity_dimension(
    "order_wt",
    cudf.Series(orders_md[n_depot:]),
    cudf.Series(vehicle_capacity_md)
)
data_model.set_order_time_windows(
    cudf.Series(order_tw_earliest_md[n_depot:]),
    cudf.Series(order_tw_latest_md[n_depot:]), 
)

data_model.set_order_service_times(
    cudf.Series(service_time_md[n_depot:])
)

data_model.set_vehicle_time_windows(
    cudf.Series(v_tw_earliest_md), 
    cudf.Series(v_tw_latest_md)
)

data_model.set_vehicle_locations(
    cudf.Series(vehicle_locations),
    cudf.Series(vehicle_locations)
)
# # Set minimum number of vehicles that need to be used to compute results
# data_model.set_min_vehicles(5)

<br>

## Create Solver Instance
---
The solver instance will take the data-model and return an optimized route plan. Additional configuration options are available to further customize solver behavior including: 
- The number of parallel agents (climbers) examining the solution search space
- The maximum time allotted to find a solution
- The minimum number of vehicles to be used and more

In [ ]:
# DO NOT CHANGE THIS CELL

solver_settings = routing.SolverSettings()
# set number of climbers that will try to search for an optimal path parallely
solver_settings.set_number_of_climbers(4096)
# solver will run for given time limit and it will fail if needs more time
solver_settings.set_time_limit(5)
# solver will drop infeasible orders and assgin remaining orders
solver_settings.set_drop_infeasible_orders(True)

routing_solution = routing.Solve(data_model, solver_settings)
if routing_solution.get_status() == 0:
    print("Solution Found")
else:
    print("No Solution Found")

### Let us have a look at the output

In [ ]:
# DO NOT CHANGE THIS CELL

routes_df = routing_solution.get_route().to_pandas()
routes_df[routes_df['type']=='Depot']

# Not a looking good, Eh?

In [ ]:
#n_depot_md = len(depot_df_md)

## Parse Output

In [ ]:
output = parse_output_summary(routing_solution, n_orders, n_vehicles, orders_details_md, vehicle_df_md, location_coordinates_md, distance_matrix, time_matrix, n_depot)

**NOTE:** `get_all_routes` function is expected to take 30-60 seconds as we are making API calls to OSRM server which is external.

In [ ]:
plot_map_df = prepare_data_for_maps(depot_df_md, output)
all_routes = get_all_routes(plot_map_df)

In [ ]:
# DO NOT CHANGE THIS CELL

from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
from ipywidgets import interact

w = widgets.Dropdown(
    options=list(vehicle_df_md.vehicle_id),
    value=list(vehicle_df_md.vehicle_id)[0],
    description='Vehicle ID:',
)

def on_change(value):
    if value in output.keys():
        idx = list(plot_map_df[plot_map_df['vehicle_id'] == value].index)
        filtered_dict = {k: v for k, v in all_routes.items() if k in idx}
        display(get_map_by_vehicle(filtered_dict))
        
    else:
        print("This Vehicle is not assigned to any order!!")

interact(on_change, value=w)